In [ ]:
from apiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow

CLIENT_SECRET_FILE = 'client_secret.json'
SCOPES = ['https://www.googleapis.com/auth/youtube.force-ssl']

flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
credentials = flow.run_console()
youtube = build('youtube', 'v3', credentials=credentials)


#https://www.youtube.com/channel/UCC5hntNsVE7rQRYqqv_ON0g/


In [ ]:
import pandas as pd

def show_playlist():
    request = youtube.playlists().list(
            part="snippet,contentDetails",
            mine=True,
            maxResults=50,
        )
    response = request.execute()
    v=[{ 'id': item['id'], 'title': item['snippet']['title'] ,'count': item["contentDetails"]["itemCount"]} for item in response['items']]
    print(v)
    return v
    

def show_videolist_bypage(token):
    request = youtube.search().list(
        part="snippet",
        forMine=True,
        type="video",
        maxResults=50,
        order="date",
        pageToken=token
    )
    response = request.execute()
    #print(response)
    return response

def show_videolist():
    data =pd.DataFrame()
    count =0
    size=50
    max=1
    nextPageToken =None
    while(count*size <= max) :
        rs=show_videolist_bypage(nextPageToken)
        max=int(rs['pageInfo']['totalResults'])
        nextPageToken= rs['nextPageToken']
        val=[{'id':d['id']['videoId'], 'title' :d['snippet']['title'] }  for d in rs['items']] 
        print(len(val),max,nextPageToken)
        data=data.append(val)
        count+=1
    
    return data

def create_playlist(youtube, title):
    request = youtube.playlists().insert(
        part="snippet,status",
        body={
          "snippet": {
            "title": "ML & AI |" + title,
            "description": "ML & AI |" + title,
            "tags": [
              "ML,AI "          
            ],
            "defaultLanguage": "en"
          },
          "status": {
            "privacyStatus": "Unlisted"
          }
        }
    )
    response = request.execute()
    if(response is not None and 'id' in response) :
        print("playlist added successfully with id :{0}".format(response["id"]))
    else:
          print("failed to create playlist ")
          print(response)
        
def add_playlist_item(youtube,playlist_id, playlist_item_id):
    request = youtube.playlistItems().insert(
        part="snippet",
        body={
          "snippet": {
            "playlistId": playlist_id,
            "position": 0,
            "resourceId": {
              "kind": "youtube#video",
              "videoId": playlist_item_id
            }
          }
        }
    )
    response = request.execute()
    print(response)

def delete_playlist(playlist_id):
     request =  youtube.playlists().delete(id=playlist_id)
     response = request.execute()
     print(response)
        

In [12]:
##video listing
#rs= show_videolist()
#rs.to_csv('youtube_videlist.csv')
#len(rs)
#add videolist item
#add_playlist_item(youtube,"PL9tQy71sPLeHoB-g1H08J1hR8ZsXWzz7Z","05yT5rR-XsM")

#show playlists
#rs= show_playlist()

#create playlist

#create_playlist(youtube,"")

#delete_playlist()



50 281 Cib3_sLbW_____85UGtzM1pxNmRiWQD_Af_-OVBrczNacTZkYlkAARAyIQVwDsDzTq1QOQAAAACkJD0BSAFQAloLCZJZP-HYunVREANg-cb14wc=
50 281 Cib3_sGFC_____9aN3FSLTJDS3B3dwD_Af_-WjdxUi0yQ0twd3cAARBkIQVwDsDzTq1QOQAAAAD0ej4BSAFQAloLCZJZP-HYunVREANg-cb14wc=
50 281 Cib3_sGB-P____9MYkwzWVU0dHpKUQD_Af_-TGJMM1lVNHR6SlEAARCWASEFcA7A806tUDkAAAAAB34-AUgBUAJaCwmSWT_h2Lp1URADYPnG9eMH
50 281 Cib3_r-d5P____9zQnRsRGlCbDVOYwD_Af_-c0J0bERpQmw1TmMAARDIASEFcA7A806tUDkAAAAAG2JAAUgBUAJaCwmSWT_h2Lp1URADYPnG9eMH
50 281 Cib3_r-aIP____9PV2k5QlFFVlhGawD_Af_-T1dpOUJRRVZYRmsAARD6ASEFcA7A806tUDkAAAAA32VAAUgCUAJaCwmSWT_h2Lp1URADYPnG9eMH
31 281 Cib3_r-Ylf____9LSEJKZUFoVW5VMAD_Af_-S0hCSmVBaFVuVTAAARCZAiEFcA7A806tUDkAAAAAamdAAUgBUAJaCwmSWT_h2Lp1URADYPnG9eMH


281

In [55]:
import os
import pandas as pd

def get_video_files(fullpath):
    filter_extension=".ts"
    data = [ { 'playlist' : '|'.join(path.split('\\')[6:]) , 'file_name':os.path.splitext(file)[0],  "path" : path}  for path, subdirs, files in os.walk(fullpath) for file in files if file.endswith(filter_extension)]
    df= pd.DataFrame(data)
    return df

def create_playlist_data(fullpath):
    df =get_video_files(fullpath)
    return groups.groups.keys()

In [58]:

df=get_video_files(r"C:\Users\Ashwini\Downloads\PGDM\Video")
#df
df.to_csv('dir_file_status.csv')
    
#dt=create_playlist_data(r"C:\Users\Ashwini\Downloads\PGDM\Video")

#for g in dt:
#    print(g)
    #create_playlist(youtube,g)
#print(len(dt))


In [ ]:
#show_playlist(youtube)

In [ ]:
from googleapiclient.http import MediaFileUpload


def upload_video(youtube,file_path):
    request = youtube.videos().insert(
        part="snippet,status",
        body={
          "snippet": {
            "categoryId" :27, #education
            "tags": "Machine Learning and AI,API call", 
            "description": "",
            "title": "Test video upload."
          },
          "status": {
            "privacyStatus": "Unlisted"
          }
        },
        
        
        media_body=MediaFileUpload(file_path)
    )
    response = request.execute()
    
    if response is not None and 'id' in response:
          print("Video id {0} was successfully uploaded.".format(response['id']))
    else:
        print("The upload failed with an unexpected response: "+  file_path)
        
    #print(response)
    #https://developers.google.com/youtube/v3/guides/uploading_a_video

In [ ]:
path=r"C:\Users\Ashwini\Downloads\PGDM\Video\Course 1\M-1 (Intro)\Graphical Models-278020.ts"

#upload_video(youtube,path)

In [62]:
import pandas as pd

file_status = pd.read_csv('dir_file_status.csv')
youtube_videlist = pd.read_csv('youtube_videlist.csv')

del file_status['path'],file_status['playlist']

file_status['file_name']= file_status['file_name'] \
.str.replace('-',' ').str.replace('(','').str.replace(')','').str.replace('.',' ')


print(len(file_status['file_name']),len(youtube_videlist['title']))

rs1=file_status.merge(youtube_videlist,left_on='file_name', right_on='title', how='left')

#rs1.to_csv('result.csv')

#file_status['file_name']==youtube_videlist['title']

rs1[rs1['id'].isnull()].to_csv('not_uploaded.csv')


196 281
